# Ejercicio 4
La idea principal de este ejercicio es estudiar si en las redes propuestas (ojo, la red de colaboraciones es una red pesada) existe asortatividad en el grado. Más en general, comprender cuál es el grado medio de los vecinos de un nodo, en función del grado de este nodo. A su vez, se pide que se repita el análisis para las redes de proteínas vistas anteriormente.

### Inciso (a)
Para este inciso, es importante entender los pasos necesarios para llevar acabo la tarea pedida. Entendemos que, en primer lugar, es recomendable trabajar con el diccionario de nodos y sus respesctivos grados antes que una lista de grados (pensar, de forma alternativa, si puede ser útil trabajar con un diccionario cuyas llaves -keys- sean los distintos grados y los valores -values- listas de nodos con determinado grado). A su vez, para tener acceso a los vecinos de un nodo, podemos hacer uso de la matriz de adyacencia de la red, pero también tenemos una función de la librería networkx que nos permite acceder a un iterable con los vecinos de determinado nodo:



```
# Opción 1
vecinos_nodo_i = Red.neighbors(i) # donde Red es un nx.Graph() e i un nodo cualquiera

# Opción 2
vecinos_nodo_i = Red[i] # ojo porque acá obtenemos un diccionario donde podemso tener información sobre el enlace entre el nodo i y sus vecinos (por ejemplo el peso)
```
La idea final es que estudiemos la relación entre el grado medio de los vecinos de los nodos de grado k en función de k. El estudio de esta relación se debe hacer en base a los dos modelos propuestos (Newman y Barabasai) sobre el origen de la asortatividad.



### i.


In [1]:
import numpy as np
Red_netscience = nx.read_gml('./data_01/netscience.gml')
Red_asjuly06 = nx.read_gml('./data_01/as-22july06.gml')

NameError: name 'nx' is not defined

In [ ]:
def asort(Red,title):
    asortatividad={} 
    for nodo_i in Red.nodes():
        #Calculo grado del nodo i
        grado_i=Red.degree[nodo_i]

        grado_vecinos_i=[]
        vecinos_nodo_i = Red.neighbors(nodo_i) # G.neighbors(node) devuelve un type iterator
        for vecino in vecinos_nodo_i:
          grado_vecinos_i.append(Red.degree[vecino])

        if len(grado_vecinos_i)==0: #no podemos hacer la media de un array vacio
            grado_medio_vecinos_i=0
        else:
            grado_medio_vecinos_i=np.mean(grado_vecinos_i)

        #Si ya existe la key con el grado, le appendeo el nuevo valor medio
        if grado_i in asortatividad:
            asortatividad[grado_i].append(grado_medio_vecinos_i)
        #Si no, creo la key
        else:	
            asortatividad[grado_i]=[grado_medio_vecinos_i]
    #Para cada grado calculo el promedio del promedio de los grados de los vecinos de cada nodo
    for grado in asortatividad:
        asortatividad[grado]=np.mean(asortatividad[grado])

    lists= sorted(asortatividad.items()) # sorted by key, return a list of tuples
    x, y = zip(*lists) # unpack a list of pairs into two tuples

    #Saco el grado 0 si es que existe porque no aporta informacion y complica el analisis posterior con logaritmos
    if x[0]==0:
        x, y = x[1:],y[1:]
      
    plt.figure()
    plt.xlabel('Grado k')
    plt.ylabel('knn(k)')
    plt.title(title)
    plt.plot(x,y)
    plt.show()

    return(x,y)

In [ ]:
asortatividad_netscience=asort(Red_netscience,'netscience')
asortatividad_asjuly06=asort(Red_asjuly06,'asjuly06')

### ii.
En el caso de la red netscience tenemos asortatividad dado que nodos de bajo grado se relacionan con nodos de bajo grado, y el valor medio del grado de los vecinos (knn(k)) se va incrementando a medida que aumenta el grado. 

Contrariamente, en el caso de la red asjuly06 tenemos disasortatividad dado que nodos de bajo grado se relacionan con nodos de muy alto grado y el valor medio del grado de los vecinos (knn(k)) se va reuciendo a medida que aumenta el grado.

### iii.

In [ ]:
def exponente_correlacion(asortatividad):
    z=np.polyfit(np.log(asortatividad[0]), np.log(asortatividad[1]), 1)
    p=np.poly1d(z)
    mu=z[0]
    print(f"z es {z}")
    error_mu = z[1]
    #devuelve mu (factor de correlacion) y el fit lineal p
    return(mu,p,error_mu)

def fit_lineal_log(asortatividad,title):
    x_log=np.log(asortatividad[0])
    y_log=np.log(asortatividad[1])
    mu,p,error_mu=exponente_correlacion(asortatividad)
    fit=p(x_log)
    plt.plot(x_log,y_log, label='log(knn(k))')
    
    plt.plot(x_log,fit,label=f'fit mu={str(round(mu,2))+"+/-"+str(round(error_mu,3)+0.05)}')
    plt.xlabel('log(k)')
    plt.title(f'Red: {title}')
    plt.legend(loc='best')
    plt.show()
    return(mu)

In [ ]:
fit_lineal_log(asortatividad_netscience,'netscience')
fit_lineal_log(asortatividad_asjuly06,'asjuly06')

El valor de mu es positivo para la red netscience que es asortativa, y negativo para la red asjuly06 que presenta disortatividad. 

### iv.

In [ ]:
#http://math.sjtu.edu.cn/faculty/xiaodong/course/Networks%20An%20introduction.pdf
#Aca usamos la formula 8.28 y 8.29
def r_correlacion(red):
     
    Se, S1, S2, S3=0, 0, 0, 0    
    for enlace in red.edges():
        nodo_i=enlace[0]
        nodo_j=enlace[1]
        #Sumatoria sobre los enlaces distintos, de 2 ki kj
        Se += 2*red.degree[nodo_i]*red.degree[nodo_j] 

    for nodo in red.nodes():
        S1+=red.degree[nodo]
        S2+=red.degree[nodo]**2
        S3+=red.degree[nodo]**3

    r= (S1*Se - S2*S2)/ (S1*S3-S2*S2)
    
    return(r)

In [ ]:
print(f'Red netscience: r={r_correlacion(Red_netscience)}')
print(f'Red asjuly06: r={r_correlacion(Red_asjuly06)}')

El valor de r es positivo para la red netscience que es asortativa, y negativo para la red asjuly06 que presenta disortatividad. 

Teniendo en cuenta que la red netscience corresponde a interacciones entre individuos de la comunidad cientifica y la red asjuly06 a las interacciones entre sistemas autonomos en internet, estos resultados coinciden con los recopilados en el libro de Newman:

<em>In Table 8.1 we show the values of r for a range of networks and the results reveal an interesting pattern.  A pesar de que ninguno de los valores es de una magnitud grande (las correlaciones entre los grados no son particularmente fuertes), hay una clara tendencia de las redes sociales a tener un valor r positivo. Esto indica un comportamento asortativo entre los nodos similares. Por otro lado, el resto de las redes (biológicas,tencológicas,informáticas) tienen un valor de r negativo, lo que indica que los nodos de grados altos tienden a enlazarse con nodos de grados más bajos.</em>

## Inciso (b)
Repetir lo anterior pero para las redes de proteínas

In [ ]:
def analisis_asortatividad(G):
    asortatividad=asort(G,red)
    mu=fit_lineal_log(asortatividad,red)
    r=r_correlacion(G)
    return(mu,r)

asortatividad_redes_proteinas={}

for red in data:
    mu,r = analisis_asortatividad(data[red]["Grafo"])
    asortatividad_redes_proteinas[red]={}
    asortatividad_redes_proteinas[red]['mu']=mu
    asortatividad_redes_proteinas[red]['r']=r
    
pd.DataFrame(asortatividad_redes_proteinas)

En el caso de las redes de proteinas AP_MS y LIT el comportamiento es asortativo, y en el caso de la red Y2H es disortativo. La explicacion en el caso de la res AP_MS podria venir del siguiente parrafo del libro de Newman:

<em> Si una red está dividida en pequeños subgrupos unidos a vértices que están mayormente desconectados del resto de la red, entonces, como dijimos, los vértices de estos pequeños subrgrupos tienden a tener un menor grado que los vértices en subgrupos más grandes. Pero además como los miembros de los grupos pequeños comparten enlaces con miembros de ese mismo grupo,  termina sucediendo que los vértices de bajos grados van a estar unidos a otros vertices de grados bajos también. Análogamente ocurrirá con los de grado alto. Esta simple idea, al poder ser cuantificada en ciertas circunstancias, produce valores positivos de r.
    
Una posible explicación de estos patrones generados por el valor r es que la mayoría de las redes son naturalmente disasortativas por grado ya que son grafos simples. En cambio las redes sociales (y tal vez algunas otras más) anulan este sesgo natural y se vuelven asortativas según las condiciones de la estructura del grupo.
 </em>

En efecto la red AP_MS corresponde a estructuras de grupos debido a que identifica conjuntos de proteinas y no interacciones uno a uno de proteinas, en este sentido era esperable que el analisis de la asortatividad de la red demostrara en cierta manera su estructura de grupos.